In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

In [2]:
df = pd.read_parquet('Data/training_feature_v1.parquet')
label = df.iloc[:, -1].values
mask = (label == 1)

In [4]:
from sklearn.model_selection import train_test_split

idx = np.arange(len(label))

X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(idx, y, test_size=0.2, stratify=y, random_state=42)
print(X_train)

NameError: name 'y' is not defined

## Lambda

In [69]:
gain_la = pd.read_csv('feat_v4/gain.csv', index_col=0)
gain_la.shape

(3442, 15)

In [77]:
res = {}

for i in range(15):
    rank = gain_la.iloc[:, i].nlargest(100).index
    for item in rank:
        if item not in res:
            res[item] = 0
        res[item] += 1
        
res = pd.DataFrame(res.items(), columns=['feature', 'rank'])
res.query('rank >= 15').shape

(32, 2)

In [60]:
seeds = [311, 788, 2789, 4248, 8866]

In [61]:
pos_shaps = {}

for seed in seeds:
    shap_df = pd.read_parquet(f'feat_v4/shap/shap{seed}.parquet')
    pos_shap_df = shap_df[mask]
    pos_shap = pos_shap_df.abs().mean(axis=0)
    pos_shaps[seed] = pos_shap
pos_shap_df = pd.DataFrame(pos_shaps)
pos_shap_df

,311,788,2789,4248,8866
技術指標_月RSI(10),3.604608,18.036510,7.217467,14.431575,10.828987
技術指標_乖離率(20日),0.332735,1.628264,0.657585,1.302511,0.973430
技術指標_W%R(5),0.022073,0.106340,0.043410,0.085919,0.063876
主力券商_前19天分點進出,0.000379,0.000853,0.000660,0.000660,0.000660
個股券商分點區域分析_近1日賣超2分點區域密度,0.004669,0.018654,0.008708,0.015537,0.011943
...,...,...,...,...,...
買超第13名分點前9天賣均張,0.005097,0.022286,0.009507,0.018254,0.013640
賣超第4名分點前11天買筆數,0.000930,0.002544,0.000930,0.002544,0.002113
賣超第12名分點前6天賣筆數,0.001799,0.002674,0.001689,0.002241,0.001689
買超第13名分點前13天金額增減(千),0.005041,0.008590,0.006720,0.008590,0.007400


In [67]:
pos_shap_df.to_csv('feat_v4/pos_shap_abs_mean.csv')

In [65]:
table = {
    'num': [],
    'top': [],
    'cum': [],
}

for top in range(500, 2001, 500):
    res = {}
    for i in range(5):
        rank = pos_shap_df.iloc[:, i].nlargest(top).index
        for item in rank:
            if item not in res:
                res[item] = 0
            res[item] += 1
    res = pd.DataFrame(res.items(), columns=['feature', 'rank'])
    for t in range(1, 6):
        num = len(res.query(f'rank >= {t}'))
        table['num'].append(num)
        table['top'].append(top)
        table['cum'].append(t)

table = pd.DataFrame(table)
table.sort_values('num', ascending=False)

,num,top,cum
15,2331,2000,1
16,2139,2000,2
17,1999,2000,3
18,1860,2000,4
10,1794,1500,1
19,1671,2000,5
11,1606,1500,2
12,1479,1500,3
13,1386,1500,4
14,1235,1500,5


In [66]:
table.sort_values('num', ascending=False).to_csv('feat_v4/pos_shap_top_cum.csv', index=False)

## Feature Select

In [ ]:
gain = pd.read_csv('full_feat/fi_gain.csv', index_col=0)
gain

In [ ]:
res = {}

for i in range(15):
    rank = gain.iloc[:, i].nlargest(1500).index
    for item in rank:
        if item not in res:
            res[item] = 0
        res[item] += 1
        
res = pd.DataFrame(res.items(), columns=['feature', 'rank'])
res.query('rank >= 11').shape

In [ ]:
table = {
    'num': [],
    'top': [],
    'cum': [],
}

for top in range(500, 3001, 500):
    res = {}
    for i in range(15):
        rank = gain.iloc[:, i].nlargest(top).index
        for item in rank:
            if item not in res:
                res[item] = 0
            res[item] += 1
    res = pd.DataFrame(res.items(), columns=['feature', 'rank'])
    for t in range(7, 16):
        num = len(res.query(f'rank >= {t}'))
        table['num'].append(num)
        table['top'].append(top)
        table['cum'].append(t)

table = pd.DataFrame(table)
table.sort_values('num', ascending=False).to_csv('full_feat/fi_rank.csv', index=False)

In [ ]:
shap_df = pd.read_parquet('feature_select/shap_values.parquet')
new_shap_df = shap_df / 5
new_shap_df

In [ ]:
feature_si = pd.DataFrame({
    'feature': new_shap_df.columns,
    'sum': new_shap_df.sum().values,
    'abs_sum': new_shap_df.abs().sum().values,
    'mean': new_shap_df.mean().values,
    'abs_mean': new_shap_df.abs().mean().values,
    'std': new_shap_df.std().values,
    'abs_std': new_shap_df.abs().std().values,
    'medain': new_shap_df.median().values,
    'abs_medain': new_shap_df.abs().median().values,
})
feature_si

In [ ]:
y = df.iloc[:, -1]
pos_mask = (y == 1)
pos_shap_df = new_shap_df[pos_mask]
pos_shap_df

In [ ]:
pos_feature_si = pd.DataFrame({
    'feature': pos_shap_df.columns,
    'sum': pos_shap_df.sum().values,
    'abs_sum': pos_shap_df.abs().sum().values,
    'mean': pos_shap_df.mean().values,
    'abs_mean': pos_shap_df.abs().mean().values,
    'std': pos_shap_df.std().values,
    'abs_std': pos_shap_df.abs().std().values,
    'medain': pos_shap_df.median().values,
    'abs_medain': pos_shap_df.abs().median().values,
})

# fi_shap_pos
pos_feature_si

## top 3000
------

### Gain

In [ ]:
gain = pd.read_csv(filepath_or_buffer='top_3000/fi_gain.csv', index_col=0)
gain.shape

In [ ]:
res = {}

for i in range(15):
    rank = gain.iloc[:, i].nlargest(500).index
    for item in rank:
        if item not in res:
            res[item] = 0
        res[item] += 1
        
res = pd.DataFrame(res.items(), columns=['feature', 'rank'])
res.query('rank >= 6').shape

### Shap

In [ ]:
shap_df = pd.read_parquet('top_3000/fi_shap.parquet')
new_shap_df = shap_df / 5
new_shap_df

In [ ]:
feature_si = pd.DataFrame({
    'feature': new_shap_df.columns,
    'mean': new_shap_df.mean().values,
    'abs_mean': new_shap_df.abs().mean().values,
    'sum': new_shap_df.sum().values,
    'abs_sum': new_shap_df.abs().sum().values,
    'std': new_shap_df.std().values,
    'abs_std': new_shap_df.abs().std().values,
    'medain': new_shap_df.median().values,
    'abs_medain': new_shap_df.abs().median().values,
})
feature_si

In [ ]:
feature_si.sort_values(by='abs_mean', ascending=False)

In [ ]:
label = df.iloc[:, -1]
pos_mask = (label == 1)
neg_mask = (label == 0)


In [ ]:
neg_shap_df = new_shap_df[neg_mask]
neg_shap_df

In [ ]:
neg_shap_df.abs().mean(axis=1).sort_values(ascending=False)